MachineLeaningCourse vol.4
# 正則化回帰

今回は、これまで勉強してきた回帰モデルに対して、正則化を適用することによって「過学習」を抑制する手法を説明していく。


### 正則化とは？
簡単に言うと、過学習を回避するためのテクニックです。

例えば、モデルに５００個の特徴量を入れて学習したとき、５００個のパラメータがコスト関数を最小化させる解として計算されます。

このときにパラメータが多すぎると、テストデータだけににフィットさせる為にたまたま現れてしまう汎化性のないパターンを学習されてしまい、過学習の原因となります。

それの対処法としてあげられるのが、「正則化」です。

$$Regularization = \frac{1}{n} \sum^{n}_{i=0} (y^{(i)}-\hat{y}^{(i)})^2　+ f(\beta)$$

普通のモデルであれば、二乗誤差を最小化するだけですが、正則化を用いると一緒に $f(\beta)$も最小化します。

イメージがつきやすい言い方にすると、「誤差と一緒にパラメーターの絶対値も最小化しよう」という感じです。

$f(\beta)$に入る式によって、正則化回帰の種類が分かれます。
- リッジ回帰
- LASSO
- ElasticNet

の３つを今回は説明していきます。

### リッジ回帰
リッジ回帰はL2ペナルティ付きのモデルである。このモデルの式では、最小二乗コスト関数に対して重みの平方和を足し合わせる。

$$Ridge = \sum^{n}_{i=0} (y^{(i)}-\hat{y}^{(i)})^2　+ \lambda\sum^{m}_{j=1} w^2_j$$

リッジ回帰は、式の通りで $w$の大きさに制限を加えています。

モデル構築用データセットの目的変数の誤差の二乗和のみを小さくするのではなく、回帰係数の二乗和も一緒に小さくしています。つまり、

~~~
{誤差の二乗和}＋λ{回帰係数の二乗和}
~~~

を最小化します。これによって、回帰係数が大きくなってモデルが複雑になることを防ぎます。

最小化するときの、誤差の二乗和の項に対する回帰係数の二乗和の項の比率である$\lambda$の値は解析者が決める必要があります。

$\lambda$の値を大きくすると、正則化の強さを引き上げ、モデルの重みを小さくします。

一つ注意点としては、切片項である$w_0$は正則化の対象としないことをあげておく。


リッジ回帰の効果をまとめると、__「L2ペナルティーにより係数を縮小して過学習を抑える」__となる。

では、前回と同様にbostonデータセットで学習していきましょう。


In [83]:
# データセットの読み込み
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.cross_validation import train_test_split

# データセットを読み込み
housing = load_boston()
columns = housing.feature_names

# Pandasのデータフレームに変換
boston = pd.DataFrame(housing.data, columns=columns)

# 目的変数をDataFrameへ変換
boston['MEDV'] = np.array(housing.target)
columns = list(columns)
columns.append('MEDV')

# 説明変数
X = boston.loc[:, housing.feature_names].values
# 目的変数
y = boston.loc[:, 'MEDV'].values

# トレーニングデータ：７割、テストデータ：３割にデータを分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

それでは、scikit-learnのRidge回帰を行いましょう。

【公式ドキュメント】
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

In [84]:
from sklearn.linear_model import Ridge

# ここでのalphaは数式のλと同じです
model= Ridge(alpha=1.0)
model.fit(X_train, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [85]:
print('R^2 Train : %.3f, Test : %.3f' % (model.score(X_train, y_train), model.score(X_test, y_test)))

R^2 Train : 0.762, Test : 0.667


alpha=1.0の場合は上記の様なスコアになりました。

今度は、alphaの値を変えて最適な値を推定しましょう。

In [86]:
# alphaの数値を0.00001~10000まで変えてR^2を求める

for i in range(-5, 5):
    al = 10 ** i
    model= Ridge(alpha=al)
    model.fit(X_train, y_train)
    print('alpha : %f, R^2 Train : %.3f, Test : %.3f' % (al, model.score(X_train, y_train), model.score(X_test, y_test)))

alpha : 0.000010, R^2 Train : 0.764, Test : 0.674
alpha : 0.000100, R^2 Train : 0.764, Test : 0.674
alpha : 0.001000, R^2 Train : 0.764, Test : 0.674
alpha : 0.010000, R^2 Train : 0.764, Test : 0.673
alpha : 0.100000, R^2 Train : 0.764, Test : 0.673
alpha : 1.000000, R^2 Train : 0.762, Test : 0.667
alpha : 10.000000, R^2 Train : 0.756, Test : 0.657
alpha : 100.000000, R^2 Train : 0.739, Test : 0.642
alpha : 1000.000000, R^2 Train : 0.687, Test : 0.600
alpha : 10000.000000, R^2 Train : 0.556, Test : 0.482


最も良い結果をだしたのは、
alpha :0.001, 0.0001, 0.00001での

~~~
R^2 Train : 0.764, Test : 0.674
~~~
という結果ですね。

alpha : 1.0(default)の場合は

~~~
R^2 Train : 0.762, Test : 0.667
~~~
でしたので、少しスコアが改善しています。


では、重み（ｗ）を見てみましょう。

In [87]:
# 重み
from sklearn.linear_model import LinearRegression
model= LinearRegression()
model.fit(X_train, y_train)
print("線形回帰の重み(w)")
print(model.coef_)
print(model.intercept_)
print("")
model= Ridge(alpha=0.001)
model.fit(X_train, y_train)
print("Ridge回帰(alpha=0.001の重み(w)")
print(model.coef_)
print(model.intercept_)

線形回帰の重み(w)
[ -1.19858618e-01   4.44233009e-02   1.18612465e-02   2.51295058e+00
  -1.62710374e+01   3.84909910e+00  -9.85471557e-03  -1.50002715e+00
   2.41507916e-01  -1.10671867e-02  -1.01897720e+00   6.95273216e-03
  -4.88110587e-01]
37.9925927703

Ridge回帰(alpha=0.001の重み(w)
[ -1.19852241e-01   4.44264240e-02   1.17974944e-02   2.51295892e+00
  -1.62553136e+01   3.84921593e+00  -9.87075198e-03  -1.49982374e+00
   2.41458946e-01  -1.10682078e-02  -1.01880481e+00   6.95361177e-03
  -4.88121821e-01]
37.9816068082


bostonデータセットはデータ量・特徴量ともに少ない方なので正則化の恩恵が少ないようですね笑

### LASSO
LASSOはL1ペナルティ付きのモデルである。このモデルの式では、モデル構築用データセットの目的変数の誤差の二乗和と一緒に、回帰係数の絶対値の和も小さくしています。

$$LASSO = \frac{1}{2n} \sum^{n}_{i=0} (y^{(i)}-\hat{y}^{(i)})^2　+ \lambda\sum^{m}_{j=1} |w_j|$$

つまり、
~~~
{誤差の二乗和}＋λ{回帰係数の絶対値の和)}
~~~
を最小化します。

LASSOの効果をまとめると、__「L1ペナルティーにより変数選択と次元削減を行う」__となります。

それでは、scikit-learnのLASSOを行いましょう。

【公式ドキュメント】
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

In [88]:
from sklearn.linear_model import Lasso

model= Lasso(alpha=1.0)
model.fit(X_train, y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [89]:
print('R^2 Train : %.3f, Test : %.3f' % (model.score(X_train, y_train), model.score(X_test, y_test)))

R^2 Train : 0.708, Test : 0.611


alpha=1.0の場合は上記の様なスコアになりました。

リッジ回帰と同様に、alphaの値を変えて最適な値を推定しましょう。

In [90]:
# alphaの数値を0.00001~10000まで変えてR^2を求める

for i in range(-5, 5):
    al = 10 ** i
    model= Lasso(alpha=al)
    model.fit(X_train, y_train)
    print('alpha : %f, R^2 Train : %.3f, Test : %.3f' % (al, model.score(X_train, y_train), model.score(X_test, y_test)))

alpha : 0.000010, R^2 Train : 0.764, Test : 0.674
alpha : 0.000100, R^2 Train : 0.764, Test : 0.674
alpha : 0.001000, R^2 Train : 0.764, Test : 0.673
alpha : 0.010000, R^2 Train : 0.764, Test : 0.671
alpha : 0.100000, R^2 Train : 0.753, Test : 0.653
alpha : 1.000000, R^2 Train : 0.708, Test : 0.611
alpha : 10.000000, R^2 Train : 0.538, Test : 0.483
alpha : 100.000000, R^2 Train : 0.235, Test : 0.196
alpha : 1000.000000, R^2 Train : 0.000, Test : -0.006
alpha : 10000.000000, R^2 Train : 0.000, Test : -0.006


最も良い結果をだしたのは、
alpha :0.0001, 0.00001での

~~~
R^2 Train : 0.764, Test : 0.674
~~~
という結果ですね。

alpha : 1.0(default)の場合は

~~~
R^2 Train :　0.708, Test : 0.611
~~~
でしたので、少しスコアが改善しています。

In [91]:
# 重み
model= LinearRegression()
model.fit(X_train, y_train)
print("線形回帰の重み(w)")
print(model.coef_)
print(model.intercept_)
print("")
model= Lasso(alpha=0.0001)
model.fit(X_train, y_train)
print("Ridge回帰(alpha=0.001の重み(w)")
print(model.coef_)
print(model.intercept_)

線形回帰の重み(w)
[ -1.19858618e-01   4.44233009e-02   1.18612465e-02   2.51295058e+00
  -1.62710374e+01   3.84909910e+00  -9.85471557e-03  -1.50002715e+00
   2.41507916e-01  -1.10671867e-02  -1.01897720e+00   6.95273216e-03
  -4.88110587e-01]
37.9925927703

Ridge回帰(alpha=0.001の重み(w)
[ -1.19843008e-01   4.44292496e-02   1.17197651e-02   2.51157621e+00
  -1.62363271e+01   3.84913623e+00  -9.88538154e-03  -1.49953756e+00
   2.41405637e-01  -1.10696768e-02  -1.01860377e+00   6.95484627e-03
  -4.88155690e-01]
37.9697218102


### Elastic Net
ElasticNetは、リッジ回帰とLASSOの折衷案である。

$$ElasticNet = \frac{1}{2n} \sum^{n}_{i=0} (y^{(i)}-\hat{y}^{(i)})^2　+ \lambda_1\sum^{m}_{j=1} w^2_j  +  \lambda_2\sum^{m}_{j=1} |w_j|$$

つまり、

~~~
{誤差の二乗和}＋λ1{回帰係数の二乗和}＋λ2{回帰係数の絶対値の和}
~~~
となる。


それでは、scikit-learnのElasticNetを行いましょう。

scikir-learnでは以下の式を最小化する。

$$ElasticNet = \frac{1}{2n} \sum^{n}_{i=0} (y^{(i)}-\hat{y}^{(i)})^2　+ \alpha *L_1ratio\sum^{m}_{j=1} |w_j|  +  \frac{1}{2} \alpha* (1 - L_1ratio)\sum^{m}_{j=1} w^2_j$$

【公式ドキュメント】
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html

In [92]:
from sklearn.linear_model import ElasticNet

model= ElasticNet(alpha=1.0, l1_ratio=0.5)
model.fit(X_train, y_train)

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [93]:
print('R^2 Train : %.3f, Test : %.3f' % (model.score(X_train, y_train), model.score(X_test, y_test)))

R^2 Train : 0.712, Test : 0.617


alphaと、l1_ratioの値を変えて最適な値を推定しましょう。

In [94]:
for i in range(-5, 5):
    for j in range(1,10):
        al = 10 ** i
        l1 = j / 10
        model= ElasticNet(alpha=al, l1_ratio=l1)
        model.fit(X_train, y_train)
        print('alpha : %f, l1 : %f,  R^2 Train : %.3f, Test : %.3f' % (al, l1,  model.score(X_train, y_train), model.score(X_test, y_test)))

alpha : 0.000010, l1 : 0.100000,  R^2 Train : 0.764, Test : 0.674
alpha : 0.000010, l1 : 0.200000,  R^2 Train : 0.764, Test : 0.674
alpha : 0.000010, l1 : 0.300000,  R^2 Train : 0.764, Test : 0.674
alpha : 0.000010, l1 : 0.400000,  R^2 Train : 0.764, Test : 0.674
alpha : 0.000010, l1 : 0.500000,  R^2 Train : 0.764, Test : 0.674
alpha : 0.000010, l1 : 0.600000,  R^2 Train : 0.764, Test : 0.674
alpha : 0.000010, l1 : 0.700000,  R^2 Train : 0.764, Test : 0.674
alpha : 0.000010, l1 : 0.800000,  R^2 Train : 0.764, Test : 0.674
alpha : 0.000010, l1 : 0.900000,  R^2 Train : 0.764, Test : 0.674
alpha : 0.000100, l1 : 0.100000,  R^2 Train : 0.764, Test : 0.673
alpha : 0.000100, l1 : 0.200000,  R^2 Train : 0.764, Test : 0.673
alpha : 0.000100, l1 : 0.300000,  R^2 Train : 0.764, Test : 0.673
alpha : 0.000100, l1 : 0.400000,  R^2 Train : 0.764, Test : 0.673
alpha : 0.000100, l1 : 0.500000,  R^2 Train : 0.764, Test : 0.673
alpha : 0.000100, l1 : 0.600000,  R^2 Train : 0.764, Test : 0.673
alpha : 0.

最高結果は、

~~~
R^2 Train : 0.764, Test : 0.674
~~~
となりました。



どうやら、このデータセットでは正則化しない方が精度がよさげですね。

次は、非線形関係をモデリングすることに関して勉強していきます。